# Datos entreneamiento Shift

En este notebook vamos a aplicar el shift para las obtener las columnas mas correlacionadas.

- __Datos recibidos:__ obtenidos de notebook __1-datos_entrenamiento_shift.ipynb__
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Notas del proyecto__
Necesitamos generar un csv con los niveles máximos, mínimo y promedio por día, se perderá la información de la estación, también generaremos el atributo PM10mean y PM25mean que representarán los valores de las últimas 24 horas de los contaminantes.

calidad de caire
pronostico contaminación

## Pasos
[X] Aplicar Shift con Correlaciones <br>
[X] Sacar minimo promedio y maximo <br>
[X] Obtener columnas más correlacionadas<br><br>

In [49]:
contaminante = "PM2.5"

In [50]:
import pandas as pd
pd.options.mode.chained_assignment = None 

In [51]:
data_hour_merge_24 = pd.read_csv('./datos_entrenamiento_outliers.csv')

In [52]:
data_hour_merge_24[data_hour_merge_24.id_station=="BJU"].head(10)

,index,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WDR,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
3219,59121,0.9,NaN,NaN,NaN,2.0,51.0,38.0,42.0,NaN,12.2,219.0,1.5,8,2016-01-08 00:00:00,0,BJU,1,NaN,NaN
3243,58552,NaN,NaN,NaN,NaN,1.0,25.0,9.0,59.0,NaN,10.8,145.0,1.4,8,2016-01-08 01:00:00,1,BJU,1,NaN,NaN
3268,58577,0.6,NaN,NaN,NaN,2.0,16.0,7.0,53.0,NaN,10.2,172.0,1.3,8,2016-01-08 02:00:00,2,BJU,1,NaN,NaN
3293,58602,0.7,NaN,NaN,NaN,1.0,18.0,8.0,60.0,NaN,9.3,200.0,1.3,8,2016-01-08 03:00:00,3,BJU,1,NaN,NaN
3318,58627,0.6,NaN,NaN,NaN,1.0,19.0,7.0,58.0,NaN,9.2,106.0,1.2,8,2016-01-08 04:00:00,4,BJU,1,NaN,NaN
3343,58652,0.7,NaN,NaN,NaN,1.0,26.0,14.0,59.0,NaN,9.2,319.0,1.4,8,2016-01-08 05:00:00,5,BJU,1,NaN,NaN
3368,58677,0.7,NaN,NaN,NaN,1.0,24.0,12.0,61.0,NaN,9.6,78.0,1.5,8,2016-01-08 06:00:00,6,BJU,1,NaN,NaN
3393,58702,0.8,NaN,NaN,NaN,1.0,19.0,8.0,63.0,NaN,9.7,126.0,1.4,8,2016-01-08 07:00:00,7,BJU,1,NaN,NaN
3418,58727,0.6,NaN,NaN,NaN,6.0,16.0,3.0,47.0,NaN,11.2,232.0,1.9,8,2016-01-08 08:00:00,8,BJU,1,NaN,NaN
3443,58752,1.0,NaN,NaN,NaN,10.0,26.0,5.0,44.0,NaN,12.7,209.0,2.2,8,2016-01-08 09:00:00,9,BJU,1,NaN,NaN


Quedémonos sin WSR:

In [53]:
data_hour_merge_24 = data_hour_merge_24[['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', "PM2.5",'RH', 'SO2', 'TMP',
       'WSP', 'dia', 'fecha', 'hora', 'id_station', 'mes',"PM10mean","PM25mean"]].reset_index(drop=True)

Hagamos una lista con todas las estaciones:

### Arreglemos las fechas:

In [54]:
data_hour_merge_24 = data_hour_merge_24[(data_hour_merge_24.fecha > "2015-12-01 23:00:00")]
data_hour_merge_24['dia'] = data_hour_merge_24['dia'].astype(str).astype(float)
data_hour_merge_24['mes'] = data_hour_merge_24['mes'].astype(str).astype(float)

Creemos un diccionario con __data_hour_merge_24__ dividido por estación:

In [55]:
estaciones = data_hour_merge_24.id_station.unique().tolist()

In [56]:
data_est = {}
for elem in estaciones:
    data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem]

In [57]:
data_hour_merge_24 = pd.read_csv("./datos_modelos_limpios.csv")

In [58]:
data_hour_merge_24 = data_hour_merge_24.dropna(subset=[contaminante]).reset_index(drop=True)#PM25

In [59]:
data_hour_merge_24.head(10)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
0,0.9,7.0,39.0,45.0,6.0,45.0,17.0,NaN,21.0,NaN,NaN,2.0,2016-01-02 23:00:00,23,CAM,1.0,47.541667,27.458333
1,0.8,6.0,44.0,50.0,7.0,44.0,25.0,52.0,26.0,18.1,2.2,2.0,2016-01-02 23:00:00,23,HGM,1.0,39.125000,27.625000
2,0.4,4.0,19.0,22.0,21.0,22.0,12.0,57.0,7.0,17.5,1.6,2.0,2016-01-02 23:00:00,23,MGH,1.0,36.750000,24.833333
3,0.9,8.0,36.0,44.0,14.0,71.0,43.0,56.0,3.0,18.5,1.3,2.0,2016-01-02 23:00:00,23,SAG,1.0,52.208333,32.250000
4,0.8,13.0,43.0,56.0,4.0,52.0,21.0,56.0,18.0,16.6,1.6,2.0,2016-01-02 23:00:00,23,TLA,1.0,52.000000,29.291667
5,1.2,7.0,43.0,51.0,9.0,66.0,32.0,52.0,3.0,17.2,2.3,2.0,2016-01-02 23:00:00,23,XAL,1.0,53.166667,26.000000
6,0.9,6.0,29.0,34.0,4.0,18.0,4.0,NaN,13.0,NaN,NaN,3.0,2016-01-03 00:00:00,0,CAM,1.0,46.083333,26.916667
7,0.6,2.0,27.0,28.0,10.0,30.0,19.0,60.0,10.0,14.7,2.9,3.0,2016-01-03 00:00:00,0,HGM,1.0,38.500000,27.333333
8,0.5,11.0,25.0,37.0,6.0,29.0,17.0,68.0,10.0,14.1,1.7,3.0,2016-01-03 00:00:00,0,MGH,1.0,36.208333,24.541667
9,0.6,7.0,23.0,30.0,6.0,51.0,32.0,69.0,3.0,15.1,2.7,3.0,2016-01-03 00:00:00,0,SAG,1.0,52.000000,32.416667


# Leamos el archivo de correlaciones 

Este archivo se generó en el notebook: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

In [60]:
target = "PM25mean_y"

O3_corr = pd.read_csv("./correlacion/correlaciones_"+ target +".csv")
O3_corr = O3_corr.iloc[:,1:]
O3_corr.head(5)

,contaminante,valor,horas
0,hora,0.000241,0.0
1,dia,-0.022622,0.0
2,mes,-0.231276,0.0
3,PM25mean,1.000000,0.0
4,CO,0.374677,0.0


In [61]:
O3_corr.valor.min()

-0.2921055990958838

Vamos a quedarnos con las variables con corr>0.3:

- Para PM10 (O3_corr.valor > 0.48 o  O3_corr.valor < -0.48)
- Para PM10 (O3_corr.valor > 0.6 o  O3_corr.valor < -0.6)
- Para O3 (O3_corr.valor > 0.44 o  O3_corr.valor < -0.44)

In [62]:
O3_corr_filtro = O3_corr[(O3_corr.valor > 0.48) | (O3_corr.valor < -0.48) ].reset_index(drop=True)

In [63]:
O3_corr_filtro.shape

(295, 3)

Vamos a aplicar el shift para cada elemento de la tabla __O3_corr__:

In [64]:
def shit_corr(df):
    for i in range(len(O3_corr_filtro)):
        name_column = str( O3_corr_filtro.loc[i,"contaminante"] + "_" + str( O3_corr_filtro.loc[i,"horas"] ) ) 
        df[name_column] = df[O3_corr_filtro.loc[i,"contaminante"]].shift( int(float(str(O3_corr_filtro.loc[i,"horas"]))) ) 
    return df.dropna()

Aplicamos esta función para cada DF de cada estación:

In [65]:
data_est.keys()

dict_keys(['AJM', 'AJU', 'CAM', 'CCA', 'CHO', 'COY', 'CUA', 'CUT', 'FAC', 'GAM', 'HGM', 'IZT', 'LLA', 'MER', 'MGH', 'MON', 'NEZ', 'PED', 'SAG', 'TLA', 'UAX', 'UIZ', 'XAL', 'INN', 'BJU', 'MPA', 'ACO', 'ATI', 'TAH', 'TLI', 'VIF', 'SFE', 'LPR', 'SJA', 'FAR', 'SAC'])

In [66]:
data_est["CAM"][["fecha"]].head()

,fecha
2,2016-01-02 00:00:00
25,2016-01-02 01:00:00
48,2016-01-02 02:00:00
71,2016-01-02 03:00:00
94,2016-01-02 04:00:00


In [67]:
data_shift = {}
for elem in data_est:
    data_shift[elem] = shit_corr(data_est[elem])

In [68]:
df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())

In [69]:
for key in data_est:
    df_append = df_append.append(data_est[key], ignore_index=True,sort=True)

In [70]:
df_append.head(10)

,CO,NO,NO2,NO2_1.0,NO2_10.0,NO2_11.0,NO2_12.0,NO2_13.0,NO2_14.0,NO2_15.0,...,PM25mean_9.0,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
0,0.6,2.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,62.0,2.0,14.8,1.9,2.0,2016-01-02 00:00:00,0,AJM,1.0
1,0.5,2.0,18.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,86.0,12.0,11.6,1.6,2.0,2016-01-02 01:00:00,1,AJM,1.0
2,0.5,2.0,16.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,88.0,11.0,11.3,1.6,2.0,2016-01-02 02:00:00,2,AJM,1.0
3,0.5,1.0,12.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,79.0,11.0,11.4,1.9,2.0,2016-01-02 03:00:00,3,AJM,1.0
4,0.4,1.0,11.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,79.0,9.0,11.5,1.6,2.0,2016-01-02 04:00:00,4,AJM,1.0
5,0.4,1.0,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,80.0,6.0,10.8,3.1,2.0,2016-01-02 05:00:00,5,AJM,1.0
6,0.4,1.0,11.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,82.0,4.0,10.0,1.1,2.0,2016-01-02 06:00:00,6,AJM,1.0
7,0.6,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,84.0,4.0,10.6,1.4,2.0,2016-01-02 08:00:00,8,AJM,1.0
8,0.7,15.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,86.0,4.0,10.6,1.1,2.0,2016-01-02 09:00:00,9,AJM,1.0
9,0.7,15.0,15.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,81.0,5.0,11.7,2.2,2.0,2016-01-02 10:00:00,10,AJM,1.0


In [71]:
df_append[["hora","NO2_1.0","NO2_2.0"]].head(10)

,hora,NO2_1.0,NO2_2.0
0,0,NaN,NaN
1,1,24.0,NaN
2,2,18.0,24.0
3,3,16.0,18.0
4,4,12.0,16.0
5,5,11.0,12.0
6,6,9.0,11.0
7,8,11.0,9.0
8,9,NaN,11.0
9,10,19.0,NaN


In [72]:
df_append[df_append.id_station=="BJU"].tail()

,CO,NO,NO2,NO2_1.0,NO2_10.0,NO2_11.0,NO2_12.0,NO2_13.0,NO2_14.0,NO2_15.0,...,PM25mean_9.0,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
608357,0.1,NaN,10.0,8.0,21.0,24.0,22.0,20.0,29.0,29.0,...,NaN,23.0,NaN,23.7,3.9,31.0,2019-03-31 19:00:00,19,BJU,3.0
608358,0.2,NaN,12.0,10.0,22.0,21.0,24.0,22.0,20.0,29.0,...,NaN,25.0,NaN,22.4,3.0,31.0,2019-03-31 20:00:00,20,BJU,3.0
608359,0.2,NaN,20.0,12.0,24.0,22.0,21.0,24.0,22.0,20.0,...,NaN,25.0,NaN,21.7,1.4,31.0,2019-03-31 21:00:00,21,BJU,3.0
608360,0.4,NaN,28.0,20.0,17.0,24.0,22.0,21.0,24.0,22.0,...,NaN,26.0,NaN,20.4,1.1,31.0,2019-03-31 22:00:00,22,BJU,3.0
608361,0.4,NaN,36.0,28.0,15.0,17.0,24.0,22.0,21.0,24.0,...,NaN,25.0,NaN,20.0,1.1,31.0,2019-03-31 23:00:00,23,BJU,3.0


## Groupbys

In [35]:
cols = df_append.columns.tolist()

In [36]:
cols.remove("id_station")

Finalmente, aplicamos los groupbys:

In [ ]:
data_hour_merge_24_mean = df_append.groupby(['fecha',"hora","dia","mes"]).mean()
data_hour_merge_24_mean.reset_index(inplace=True)

In [ ]:
data_hour_merge_24_max = df_append.groupby(['fecha',"hora","dia","mes"]).max()
data_hour_merge_24_max.reset_index(inplace=True)

In [ ]:
data_hour_merge_24_min = df_append.groupby(['fecha',"hora","dia","mes"]).min()
data_hour_merge_24_min.reset_index(inplace=True)

## Cambiamos los nombres:

In [ ]:
max_columns = data_hour_merge_24_max.columns.tolist()
min_columns = data_hour_merge_24_min.columns.tolist()
mean_columns = data_hour_merge_24_mean.columns.tolist()

In [ ]:
for i in range(len(max_columns)):
    if max_columns[i] not in ['fecha','hora','dia','mes','id_station']:
        max_columns[i] = max_columns[i]+"_max"
        min_columns[i] = min_columns[i]+"_min"    
        mean_columns[i] = mean_columns[i]+"_mean"

In [ ]:
data_hour_merge_24_mean.columns = mean_columns
data_hour_merge_24_min.columns = min_columns
data_hour_merge_24_max.columns = max_columns

##### Unamos los df's de manera que tengamos en una solo los datos promedio, máximo y minimo por día:

In [ ]:
data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha',"hora","dia","mes"])
data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha',"hora","dia","mes"])

guardamos el archivo:

In [ ]:
data_hour_merge.to_csv("./datos_gb_" + contaminante + "_outliers.csv", sep=',', encoding='utf-8',index=False)

##### Leamos el archivo:

In [73]:
contaminante ="PM2.5"

In [75]:
data_hour_merge = pd.read_csv("./datos_gb_" + contaminante + "_outliers.csv", sep=',', encoding='utf-8')

In [76]:
print("listo")

listo


In [77]:
data_hour_merge[["fecha","PM25mean_mean","PM25mean_1.0_mean"]].head(5)

,fecha,PM25mean_mean,PM25mean_1.0_mean
0,2016-01-02 00:00:00,NaN,NaN
1,2016-01-02 01:00:00,NaN,NaN
2,2016-01-02 02:00:00,NaN,NaN
3,2016-01-02 03:00:00,NaN,NaN
4,2016-01-02 04:00:00,NaN,NaN


## Correlacionados con el target.

Ahora vamos a sacar los atributos más correlacionado con el target, el target es la columna a pronosticar, por simplicidad solo vamos a sacar la correlación con target a las 12 horas:

Generemos los target de pronóstico:

Generemos el shift de 1 a 24 horas, ahora hacia adelante

In [78]:
contaminante ="PM25mean"

In [79]:
if contaminante == 'PM10' or contaminante == 'PM2.5':
    item = contaminante+"mean_max"
else:
    item = contaminante+"_max" 

In [80]:
'PM2.5_max'

'PM2.5_max'

In [81]:
item

'PM25mean_max'

definimos el 'PM10mean_y_frcst_24'

In [82]:
data_hour_merge[str(item+"_frcst_"+str(24))] = data_hour_merge[item].shift(-24)

In [83]:
target = str(item+"_frcst_"+ str(24))

In [84]:
target

'PM25mean_max_frcst_24'

Saquemos la correlación:

In [85]:
data_corr = data_hour_merge.corr()

Guardamos los atributos mas correlacionados en una lista:

- 0.44 para PM10
- 0.4 para O3

In [86]:
variables_mas_correlacionadas_pm10_mean = data_corr[target][(data_corr[target] > 0.46) |
                                                                           (data_corr[target] < -0.46)].index.tolist()

In [87]:
len(variables_mas_correlacionadas_pm10_mean)

135

In [88]:
variables_mas_correlacionadas_pm10_mean.append("RH_mean")#PM10
variables_mas_correlacionadas_pm10_mean.append("WSP_mean")
variables_mas_correlacionadas_pm10_mean.append("TMP_mean")
variables_mas_correlacionadas_pm10_mean = list(set(variables_mas_correlacionadas_pm10_mean))

Guardemos estas variables, no pueden cambiarse una vez que los modelos estan entrenados:

In [89]:
with open("./variables+correlacionadas/vars_mas_corr_"+ contaminante +".txt", "w") as output:
    output.write(str(variables_mas_correlacionadas_pm10_mean))

In [90]:
variables_mas_correlacionadas_pm10_mean.append("fecha")

Filtremos el DF con las variables mas correlacionadas:

In [91]:
data_hour_merge = data_hour_merge[variables_mas_correlacionadas_pm10_mean]

In [92]:
for i in range(1, 24):
    col_name = str(item+"_frcst_"+str(i))
    data_hour_merge[col_name] = data_hour_merge[item].shift(-i)
    if i==range(1, 24)[-1]:
        print(i)
data_hour_merge.dropna(inplace=True)

23


Añadamos los datos de pronostico de las siguientes variables:

In [93]:
lista_frcst = ["RH_mean","WSP_mean","TMP_mean"]

In [94]:
for item in lista_frcst:
    for i in range(1, 25):
        col_name = str(item+"_frcst_"+str(i))
        data_hour_merge[col_name] = data_hour_merge[item].shift(-i)
data_hour_merge.dropna(inplace=True)

Guardemos:

In [95]:
data_hour_merge.to_csv("./datos_modelos_"+ contaminante + "ouliers.csv",sep=',', encoding='utf-8',index=False)

In [96]:
data_hour_merge.shape

(26978, 234)